In [1]:
from xbbg import blp
import pandas as pd

today = datetime.today()
d_from = today - timedelta(days=1)
d_to = yesterday + pd.DateOffset(months=1)

NameError: name 'datetime' is not defined

In [106]:
ISO = blp.bds('ACWI US Equity','LU_COUNTRY_ISO').reset_index(drop=True) 
ISO.columns = ['ISO','Name']

BBG = blp.bds('ACWI US Equity','LU_CDR_COUNTRY_CODE').reset_index(drop=True)
BBG.columns = ['BBG','Name']
BBG['Name'] = BBG['Name'].str.upper()

Mapping = BBG.set_index('Name')['BBG'].to_dict()
ISO.insert(1, 'BBG',0)
ISO['BBG'] = ISO['Name'].map(Mapping).fillna(ISO['ISO'])

W = blp.bds('ACWI US Equity','HB_GEO_CNTRY_ALLOC')
W.columns = ['Name', 'Weight']
W = W.sort_values(by='Weight', ascending=False).reset_index(drop=True)
W.insert(0, 'ISO',0)
W.insert(1, 'BBG',0)
tmp = W['Name']
W['Name'] = W['Name'].str.upper()

Mapping = ISO.set_index('Name')['ISO'].to_dict()
Mapping.update({'U.S.': 'US', 'U.K.': 'GB','Czech Republic': 'CZ','U.A.E.': 'AE'} )
W['ISO'] = W['Name'].map(Mapping).fillna(W['ISO'])

Mapping = ISO.set_index('Name')['BBG'].to_dict()
Mapping.update({'U.S.': 'US', 'U.K.': 'GB'} )
W['BBG'] = W['Name'].map(Mapping).fillna(W['BBG'])

W['Name'] = tmp
W['Weight']  = round(W['Weight'],2)
W['Cumulative'] = W['Weight'].cumsum()

W.head()

,ISO,BBG,Name,Weight,Cumulative
0,US,US,U.S.,65.12,65.12
1,JP,JN,Japan,4.79,69.91
2,GB,GB,U.K.,3.00,72.91
3,CA,CA,Canada,2.85,75.76
4,CN,CH,China,2.40,78.16


In [109]:
HL = ['JP','US','GB','CN']
HL += ['HK','DE','TW','IN','CA','AU','CH','FR','DE','KR','IE','SE','DK','BR']
HL_dict = {}

for i in HL:
    HL_dict[i] = blp.bds('JPY Curncy','CALENDAR_NON_SETTLEMENT_DATES',
                         CALENDAR_START_DATE=d_from.strftime('%Y%m%d'), CALENDAR_END_DATE=d_to.strftime('%Y%m%d'),
                         SETTLEMENT_CALENDAR_CODE=W.loc[W['ISO']==i,'BBG'].values[0])
    

In [110]:
date_range = pd.bdate_range(start=d_from, end=d_to)
df = pd.DataFrame({'Date': date_range})
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Day'] = pd.to_datetime(df['Date']).dt.day_name().str[:3]
df.reset_index(drop=True, inplace=True)


for i in ['JP','US']:

    df[i] = ''
    tmp = pd.to_datetime(HL_dict[i].iloc[:,0])
    df[df['Date'].isin(tmp)]
    df.loc[df['Date'].isin(tmp.dt.strftime('%Y-%m-%d')),i] = 1

df


,Date,Day,JP,US
0,2024-12-03,Tue,,
1,2024-12-04,Wed,,
2,2024-12-05,Thu,,
3,2024-12-06,Fri,,
4,2024-12-09,Mon,,
5,2024-12-10,Tue,,
6,2024-12-11,Wed,,
7,2024-12-12,Thu,,
8,2024-12-13,Fri,,
9,2024-12-16,Mon,,
